## Rede Neural Convolucional (CNN)

Neste notebook iremos criar nosso classificador afim resolver o problema de gênero e idade. Para isso iremos utilizar o framework [Keras](https://keras.io)

## Dependências

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import _pickle as pkl
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import normalization
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Funções auxíliares

Iremos carregar implementar algumas funções para leitura e formatação dos dados de entrada pré processados no notebook anterior.

In [6]:
DATA_NAME = ['data_%d' % i for i in range(5)]

In [13]:
def load(name):
    with open(name+'.pkl', 'rb') as f:
        return pkl.load(f)

def wrapper_data(name):
    imagens = []
    labels = []
    for data in DATA_NAME:
        obj = load(name+data)
        for x, y in zip(obj['imagens'], obj['labels']):
            imagens.append(x)
            labels.append(y)
    return np.array(imagens, dtype=np.float16),  np.array(labels, dtype=np.uint16) 

## Arquitetura Gênero

![Keras](images/test.svg)

### Modelo

In [7]:
model = Sequential()
# convolutional layers
# 3x
model.add(Conv2D(filters=96,
                 kernel_size=(7,7), # 3x3
                 activation='relu',
                 strides=(4,4),
                 padding='valid',
                 input_shape=(227,227,3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')) # pool_size=2
model.add(BatchNormalization())

model.add(Conv2D(filters=256,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=384,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2),  padding='same'))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 56, 56, 96)        14208     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 384)       885120    
__________

### Hiper-parâmetros

In [3]:
BATCH_SIZE = 50
EPOCHS = 10000
LR = 1e-3
LR_DECAY = 1e-4 / EPOCHS

In [ ]:
model.compile(loss=categorical_crossentropy, 
              optimizer=SGD(lr=LR, decay=LR_DECAY, momentum=0.9), # adam
              metrics=['accuracy'])

### Entrada 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wrapper_data('gender/'), test_size=0.15)

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

### Treino

In [ ]:
hist = model.fit(x=X_train, y=y_train, validation_split=0.1, batch_size=BATCH_SIZE, epochs=EPOCHS)
model.save('models/model_gender.h5')

print('Train loss:', model.evaluate(X_train, y_train, verbose=0))
print('Test loss:', model.evaluate(X_test, y_test, verbose=0))

## Arquitetura Idade

![Keras](images/test1.svg)

### Modelo

In [36]:
model = Sequential()
# convolutional layers
model.add(Conv2D(filters=96,
                 kernel_size=(7,7),
                 activation='relu',
                 strides=(4,4),
                 padding='valid',
                 input_shape=(227,227,3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization()) # axis=3

model.add(Conv2D(filters=256,
                 kernel_size=(5,5),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=384,
                 kernel_size=(3,3),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2),  padding='same'))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 56, 56, 96)        14208     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 28, 96)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 28, 28, 96)        384       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 256)       614656    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 384)       885120    
__________

### Hiper-parâmetros

In [2]:
BATCH_SIZE = 50
EPOCHS = 10000
LR = 1e-3
LR_DECAY = 1e-4 / EPOCHS
MOMENTUM = 0.9

In [ ]:
model.compile(loss=categorical_crossentropy, 
              optimizer=SGD(lr=LR, decay=LR_DECAY, momentum=MOMENTUM),
              metrics=['accuracy'])

### Idade Masculina

Iremos considerar apenas os dados que tem o gênero definido como masculino

### Entrada

In [37]:
X, y = wrapper_data('serializer/age-male/')
# separando os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
# transformando para one_hot
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6229, 227, 227, 3), (1100, 227, 227, 3), (6229, 8), (1100, 8))

### Treino

In [41]:
hist = model.fit(x=X_train, y=y_train, validation_split=0.1, batch_size=BATCH_SIZE, epochs=EPOCHS)
model.save('models/model_age_male.h5')

print('Train loss:', model.evaluate(X_train, y_train, verbose=0))
print('Test loss:', model.evaluate(X_test, y_test, verbose=0))

Train on 5606 samples, validate on 623 samples
Epoch 1/10000
5606/5606 [==============================] - 232s 41ms/step - loss: 2.1261 - acc: 0.2378 - val_loss: 1.8140 - val_acc: 0.3387
Epoch 2/10000
5606/5606 [==============================] - 218s 39ms/step - loss: 1.9130 - acc: 0.2808 - val_loss: 1.7712 - val_acc: 0.3628
Epoch 3/10000
5606/5606 [==============================] - 225s 40ms/step - loss: 1.8603 - acc: 0.2963 - val_loss: 1.7428 - val_acc: 0.3868
Epoch 4/10000
5606/5606 [==============================] - 220s 39ms/step - loss: 1.7914 - acc: 0.3268 - val_loss: 1.6984 - val_acc: 0.3804
Epoch 5/10000
5606/5606 [==============================] - 226s 40ms/step - loss: 1.7418 - acc: 0.3370 - val_loss: 1.5639 - val_acc: 0.4270
Epoch 6/10000
5606/5606 [==============================] - 228s 41ms/step - loss: 1.6631 - acc: 0.3733 - val_loss: 1.5348 - val_acc: 0.4575
Epoch 7/10000
5606/5606 [==============================] - 276s 49ms/step - loss: 1.6387 - acc: 0.3732 - val_loss

KeyboardInterrupt: 